In [1]:
from transformers import pipeline

generator = pipeline('text-generation', model='distilgpt2') #CONSIDER USING FULL GPT-2 OR GPT-2-SMALL

generated_text = generator("EleutherAI has", max_length=50, num_return_sequences=1)

print(generated_text[0]['generated_text'])


2024-02-20 10:13:43.138791: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


EleutherAI has a free-trade agreement that aims to curb trade and trade between the US and many others, as well as safeguard our global and global competitiveness. The partnership follows previous agreements, but the agreement was also ratified on a scale from the U.S. Congress to the U.K. Congress. We reached out and received further comments from the Secretary of State saying we would like to thank the Director, General, of the Australian Secret Service for their assistance in addressing this issue, and


In [16]:
#Loading in the csv with the top five people that are matches for our person
import pandas as pd
top5_indices = pd.read_csv('top5_indices.csv').set_index('Unnamed: 0')
top5_indices.columns = ['match_idx']

In [18]:
top5_indices

,match_idx
Unnamed: 0,
0,4
1,8
2,9
3,2
4,5


In [20]:
df = pd.read_csv("../data/okcupid_profiles.csv") 
#Need to update to use the classifier csv output rather than subsetting from the whole csv

In [21]:
df

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,...,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,...,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",...,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,59,single,f,straight,NaN,NaN,socially,never,graduated from college/university,NaN,...,"vibrant, expressive, caring optimist. i love b...",the happiest times have been when life came to...,i make an outstanding osso bucco. i am also ve...,"i am told that people notice my smile, eyes an...",i am an avid movie watcher and follow the broa...,"my family, my dog, italy, words and music!",writing my book.,"running with my dog, finishing up the work wee...",i have a dream to sing at the alconquin in nyc...,you are seeking a long term connection of shar...
59942,24,single,m,straight,fit,mostly anything,often,sometimes,working on college/university,"white, other",...,i'm nick. i never know what to write about mys...,currently finishing school for film production...,"filmmaking, photography, graphic design, web d...","dude, i don't know.","movies: hook (the greatest adventure ever!), g...",iphone contact lenses headphones camera tv rem...,i do most of my thinking on the bus to/from wo...,"bringin' home bacon, or drinking and shakin'!",when i was 18 i got a tattoo of waldo somewher...,meh if you made it this far you might as well.
59943,42,single,m,straight,average,mostly anything,not at all,never,graduated from masters program,asian,...,"hello! i enjoy traveling, watching movies, and...","i'm a civil engineer, who enjoys helping the c...",- looking at things objectively - getting thin...,i'm quiet until i get used to the environment ...,"last book: ""game change"". movies

In [117]:
#Build the prompt that we're going to use for few shot encoding with our model
#Need to investigate how essay0's with NaN (no text) are being ranked highly in algo

def construct_prompt(df, top_matches, input_prompt):
    top_subset = df.loc[top5_indices.loc[:3,'match_idx'],'essay0']
    prompt = ""
    for match in top_subset:
        if type(match) is float: 
             continue
        prompt = prompt + "\n " + "Input: " + input_prompt + "\n Output: " + match
    prompt = prompt + "\n " + "Input: " + input_prompt + "\n" + " Output: "
    return prompt
essay_prompt = construct_prompt(df ,top5_indices, df.loc[6,'essay0'])
print(essay_prompt)


 Input: life is about the little things. i love to laugh. it's easy to do when one can find beauty and humor in the ugly. this perspective makes for a more gratifying life. it's a gift. we are here to play.
 Output: hey how's it going? currently vague on the profile i know, more to come soon. looking to meet new folks outside of my circle of friends. i'm pretty responsive on the reply tip, feel free to drop a line. cheers.
 Input: life is about the little things. i love to laugh. it's easy to do when one can find beauty and humor in the ugly. this perspective makes for a more gratifying life. it's a gift. we are here to play.
 Output: my names jake. i'm a creative guy and i look for the same in others.  i'm easy going, practical and i don't have many hang ups. i appreciate life and try to live it to the fullest. i'm sober and have been for the past few years.  i love music and i play guitar. i like tons of different bands. i'm an artist and i love to paint/draw etc. and i love creativ

In [120]:
all_returned = generator(essay_prompt, do_sample=True, min_length=50, max_length = 1024, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [121]:
all_returned[0]['generated_text']

"\n Input: life is about the little things. i love to laugh. it's easy to do when one can find beauty and humor in the ugly. this perspective makes for a more gratifying life. it's a gift. we are here to play.\n Output: hey how's it going? currently vague on the profile i know, more to come soon. looking to meet new folks outside of my circle of friends. i'm pretty responsive on the reply tip, feel free to drop a line. cheers.\n Input: life is about the little things. i love to laugh. it's easy to do when one can find beauty and humor in the ugly. this perspective makes for a more gratifying life. it's a gift. we are here to play.\n Output: my names jake. i'm a creative guy and i look for the same in others.  i'm easy going, practical and i don't have many hang ups. i appreciate life and try to live it to the fullest. i'm sober and have been for the past few years.  i love music and i play guitar. i like tons of different bands. i'm an artist and i love to paint/draw etc. and i love cr